In [2]:
"""
🎯 학습 목표: FewShotChatMessagePromptTemplate을 사용한 일관된 형식의 정보 제공 시스템 구축
📚 핵심 개념: Few-shot learning, FewShotChatMessagePromptTemplate, 예시 기반 프롬프트 엔지니어링
🚀 실행 결과: 영화 정보를 항상 동일한 형식으로 제공하는 지능형 검색 시스템

📋 과제 요구사항:
- 영화 이름을 입력받아 감독, 출연진, 예산, 흥행수익, 장르, 시놉시스 정보 제공
- LLM이 **항상** 동일한 형식으로 응답하도록 예시 기반 학습 적용
- FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate 활용
"""

# 🔧 필수 모듈 임포트
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

# === 1단계: Few-shot 학습용 예시 데이터 준비 ===
# 🧠 개념: Few-shot learning은 "보여주기 > 설명하기" 원칙에 기반
# 📚 이론적 배경: LLM은 명시적 지시보다 구체적 예시를 통해 더 정확한 패턴을 학습
# 💡 실무 활용: 일관된 출력 형식, 특정 도메인 지식, 브랜드 톤앤매너 유지

examples = [  # 📌 용도: Few-shot 학습용 예제 데이터, 타입: List[Dict[str, str]]
    {
        "movie": "Inception",  # 📌 키: human 메시지의 {movie} 변수와 정확히 일치해야 함
        "info": """감독: 크리스토퍼 놀란
주요 출연진: 레오나르도 디카프리오, 조셉 고든 레빗, 엘렌 페이지
예산: $160,000,000
흥행 수익: $829,895,144
장르: SF, 액션, 스릴러
시놉시스: 꿈을 조작하는 기술을 이용해 타인의 무의식에 침투하는 산업 스파이의 이야기"""
        # 📌 키: ai 메시지의 {info} 변수와 일치, 원하는 출력 형식을 정확히 정의
    },
    {
        "movie": "Parasite",
        "info": """감독: 봉준호
주요 출연진: 송강호, 이선균, 조여정
예산: $11,400,000
흥행 수익: $263,000,000
장르: 드라마, 스릴러
시놉시스: 가난한 가족이 부잣집에 점차 스며들며 벌어지는 블랙코미디"""
    }
]

# 💡 실무 팁: 예시 데이터 설계 가이드라인
# - 예시 개수: 3-5개가 최적 (너무 많으면 토큰 낭비, 너무 적으면 패턴 학습 부족)
# - 다양성: 다양한 장르, 국가, 시대의 영화로 구성하여 일반화 능력 향상
# - 품질: 예시의 품질이 최종 출력 품질을 결정하는 가장 중요한 요소
# - 일관성: 모든 예시가 동일한 형식과 정보 구조를 유지해야 함

# === 2단계: 예시 포맷 템플릿 정의 ===
# 🧠 개념: ChatPromptTemplate.from_messages()로 Human-AI 대화 패턴 정의
# 🔧 구조 분석: 각 예시를 (human 질문, ai 답변) 형태로 변환하는 템플릿
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "영화 '{movie}'에 대해 알려줘."),  # 📌 템플릿: 사용자가 묻는 질문 형식
    ("ai", "{info}")  # 📌 템플릿: AI가 제공할 정보 형식
])

# 📌 중요 포인트: 변수명 일치성
# - example_prompt의 {movie}는 examples의 "movie" 키와 일치
# - example_prompt의 {info}는 examples의 "info" 키와 일치
# - 변수명이 다르면 KeyError 발생

# === 3단계: FewShotChatMessagePromptTemplate 구성 ===
# 🧠 개념: 예시들을 실제 프롬프트에 자동으로 삽입하는 핵심 클래스
# 🔧 동작 원리: 
#   1. examples의 각 항목을 example_prompt에 적용
#   2. 생성된 예시들을 최종 프롬프트에 순서대로 삽입
#   3. LLM이 예시 패턴을 학습하여 동일한 형식으로 응답
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,  # 📌 용도: 예시 포맷터, 타입: ChatPromptTemplate
    examples=examples  # 📌 용도: 학습시킬 예제 데이터, 타입: List[Dict]
)

# 💡 실무 팁: FewShotChatMessagePromptTemplate의 장점
# - 자동 예시 삽입: 수동으로 예시를 프롬프트에 넣을 필요 없음
# - 형식 일관성: 모든 예시가 동일한 템플릿으로 처리되어 일관성 보장
# - 동적 예시 선택: ExampleSelector와 함께 사용하면 상황에 맞는 예시 선택 가능
# - 유지보수성: 예시 추가/수정이 쉬워 지속적인 품질 개선 가능

# === 4단계: 최종 프롬프트 구성 (시스템 + 예시 + 사용자 질문) ===
# 🧠 개념: 완전한 프롬프트는 시스템 지시 + Few-shot 예시 + 실제 질문으로 구성
# 📊 프롬프트 구조:
#   1. system: AI의 역할과 행동 방식 정의
#   2. few_shot_prompt: 예시들이 자동으로 삽입됨
#   3. human: 실제 사용자 질문
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화 정보 전문가입니다. 항상 동일한 형식으로 답변하세요."),  # 📌 역할: 전문가 정체성과 일관성 강조
    few_shot_prompt,  # 📌 핵심: Few-shot 예시들이 여기에 자동 삽입
    ("human", "영화 '{movie}'에 대해 알려줘.")  # 📌 변수: 실제 사용자가 묻는 영화명
])

# 💡 실무 팁: 시스템 메시지 최적화
# - 역할 정의: "영화 정보 전문가" - 도메인 전문성 부여
# - 행동 지시: "항상 동일한 형식" - 일관성 강조로 예시 패턴 준수 강화
# - 품질 기준: "정확하고 상세한" 등 품질 기대치 명시 가능

# === 5단계: LLM 모델 초기화 ===
# 🧠 개념: ChatOpenAI 모델 설정으로 응답 품질과 일관성 제어
chat = ChatOpenAI(
    temperature=0.1  # 📌 용도: 낮은 온도로 일관된 형식 출력 보장, 타입: float
)

# 💡 실무 팁: Few-shot 학습에서의 temperature 설정
# - 0.0-0.2: 매우 일관된 형식, 예시와 거의 동일한 구조
# - 0.2-0.5: 적당한 변화와 일관성의 균형
# - 0.5+: 창의적이지만 형식 일관성 저하 위험

# === 6단계: LCEL 체인 구성 ===
# 🧠 개념: 프롬프트와 LLM을 파이프 연산자로 연결하여 실행 가능한 체인 생성
chain = final_prompt | chat  # 📌 흐름: 프롬프트 생성 → LLM 실행 → 결과 반환

# === 7단계: 실제 실행 및 테스트 ===
# 🚀 실행: 구성된 체인으로 실제 영화 정보 요청
print("=== Few-shot 학습 기반 영화 정보 시스템 실행 ===")
result = chain.invoke({"movie": "명량"})  # 📌 변수: movie 키에 영화명 전달
print("🎬 결과:")
print(result.content)

# 📊 결과 분석 및 검증
print(f"\n📏 응답 길이: {len(result.content)}자")
print("✅ 형식 일관성: 예시와 동일한 구조로 출력되었는지 확인")

# 💡 실무 확장 아이디어:
# - 다국어 지원: 영어, 일본어 영화 정보도 동일한 형식으로 제공
# - 동적 예시 선택: 장르별로 관련성 높은 예시 자동 선택
# - 정보 검증: 외부 API(TMDB, IMDb)와 연동하여 정보 정확성 검증
# - 캐싱: 자주 요청되는 영화 정보 캐싱으로 응답 속도 향상

# ⚠️ 주의사항:
# - 변수명 일치: 모든 템플릿의 변수명이 정확히 일치해야 함
# - 예시 품질: 잘못된 정보나 형식의 예시는 전체 시스템 품질 저하
# - 토큰 제한: 예시가 많을수록 토큰 사용량 증가, 비용과 성능 고려 필요
# - 저작권: 영화 정보의 저작권 및 출처 표시 고려

# 🔗 관련 학습 자료:
# - Few-shot 학습 심화: Chapter_4_Prompt_Engineering/4.1_FewShotPromptTemplate.md
# - 동적 예시 선택: Chapter_4_Prompt_Engineering/4.3_LengthBasedExampleSelector.md
# - 프롬프트 구성: Chapter_4_Prompt_Engineering/4.4_Serialization_Composition.md

=== Few-shot 학습 기반 영화 정보 시스템 실행 ===
🎬 결과:
감독: 김한민
주요 출연진: 최민식, 조진웅, 류승룡
예산: 약 $15,000,000
흥행 수익: $135,000,000
장르: 역사, 전쟁
시놉시스: 조선시대 임진왜란 중 명량 해전을 통해 일본의 대군을 상대로 한 조선 해군의 승리를 다룬 역사 영화

📏 응답 길이: 140자
✅ 형식 일관성: 예시와 동일한 구조로 출력되었는지 확인
